In [1]:
# import packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import math
from moviepy.editor import VideoFileClip
%matplotlib inline

# angle range: 30 - 90 degrees
DIRECTION_THRESHOLD_LOWER = np.pi / 4.0
DIRECTION_THRESHOLD_UPPER = np.pi / 2.0
# value range for gradient 
GRADIENT_X_THRESHOLD_LOWER = 10
GRADIENT_X_THRESHOLD_UPPER = 200
# lower bound for filterring R&G channel
# reason: yellow(255,255,0), white(255,255,255)
CHANNEL_RG_THRESHOLD_LOWER = 180
# for yellow color
CHANNEL_B_THRESHOLD_UPPER = 50
CHANNEL_S_THRESHOLD_LOWER = 90
CHANNEL_L_THRESHOLD_LOWER = 150
CHANNEL_L_THRESHOLD_UPPER = 220

# region of interest selection
Y_PERCENTILE = 0.6
X_TOP_LEFT_PERCENTILE = 0.4
X_TOP_RIGHT_PERCENTILE = 0.6
X_BOTTOM_LEFT_PERCENTILE = 0.05
X_BOTTOM_RIGHT_PERCENTILE = 0.95

In [2]:
%run utilities.ipynb

In [3]:
def mix_threshold(img, cut=False):
    """
    Get thresholded binary image
    """
    # 
    # setup for processing
    #
    height, width = get_shape_of_image(img)
    img = smooth(img)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # 
    # threshold gradient
    # 
    # apply gradient threshold on the horizontal gradient
    horizontal_binary = horizontal_threshold(gray,
        GRADIENT_X_THRESHOLD_LOWER, GRADIENT_X_THRESHOLD_UPPER)
    # apply gradient direction threshold
    angle_binary = angle_threshold(gray, 
        DIRECTION_THRESHOLD_LOWER, DIRECTION_THRESHOLD_UPPER)

    #
    # threshold R & G channels
    #
    # R & G thresholds so that yellow lanes are detected well.
    rg_binary = rg_threshold(img)
    
    # color channel thresholds
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    L = hls[:,:,1]
    S = hls[:,:,2]
    l_binary, s_binary = ls_threshold(img)

    # combine all the thresholds
    # A pixel should either be a yellowish or whiteish
    # And it should also have a gradient, as per our thresholds
    
    combined = np.zeros_like(s_binary)
    combined[((rg_binary == 1) & (l_binary == 1)) 
             | (s_binary == 1) 
             | ((horizontal_binary == 1) & (angle_binary == 1))] = 1
    if not cut:
        return combined
    return select_region_auto(combined, Y_PERCENTILE,
                              X_TOP_LEFT_PERCENTILE, 
                              X_TOP_RIGHT_PERCENTILE, 
                              X_BOTTOM_LEFT_PERCENTILE, 
                              X_BOTTOM_RIGHT_PERCENTILE)

def ls_threshold(img):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    L = hls[:,:,1]
    mask_l = np.zeros_like(L)
    mask_l[(CHANNEL_L_THRESHOLD_LOWER < L) & (L < CHANNEL_L_THRESHOLD_UPPER)] = 1
    S = hls[:,:,2]
    mask_s = np.zeros_like(S)
    mask_s[S > CHANNEL_S_THRESHOLD_LOWER] = 1
    return mask_l, mask_s

def rg_threshold(img):
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    mask = np.zeros_like(R)
    mask[(R > CHANNEL_RG_THRESHOLD_LOWER) & 
         (G > CHANNEL_RG_THRESHOLD_LOWER) & 
         ((B < 50) | (B > 180))] = 1
    return mask

def horizontal_threshold(gray, thresh_lower=0, thresh_upper=255, sobel_kernel=7):
    sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    abs_sobel = np.absolute(sobel)
    max_value = np.max(abs_sobel)
    binary_output = np.uint8(255*abs_sobel/max_value)
    mask = np.zeros_like(binary_output)
    mask[(binary_output >= thresh_lower) & (binary_output <= thresh_upper)] = 1
    return mask

def angle_threshold(gray, thresh_lower=0, thresh_upper=np.pi/2.0, sobel_kernel=5):
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobel_x = np.absolute(sobel_x)
    abs_sobel_y = np.absolute(sobel_y)
    # Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient
    direction = np.arctan2(abs_sobel_y,abs_sobel_x)
    direction = np.absolute(direction)
    # Create a binary mask where direction thresholds are met
    mask = np.zeros_like(direction)
    mask[(direction >= thresh_lower) & (direction <= thresh_upper)] = 1
    return mask

def select_region_auto(img, y_percentile, 
                       x_top_left_percentile, x_top_right_percentile,
                       x_bottom_left_percentile, x_bottom_right_percentile):
    """
    Select the region of interest automatically. 
    The basic idea for this is to capature the lower half of the picture 
    automaticaly. The idea works as camera angle is fixed and thus the 
    region of interest is always the lower half of the image. 
    """
    height = img.shape[0]
    width = img.shape[1]
    top_left = [width * x_top_left_percentile, height * y_percentile]
    top_right = [width * x_top_right_percentile, height * y_percentile]
    bottom_left = [width * x_bottom_left_percentile, height - 1]
    bottom_right = [width * x_bottom_right_percentile, height - 1]
    polygon_vertices = np.array([[top_left, top_right, bottom_right, bottom_left]], 
                                dtype=np.int32)
    return _select_polygon_region(img, polygon_vertices)

def _select_polygon_region(img, vertices):
    """
    Select the region inside the polygon defined by the vertices
    """
    def _generate_mask(img, vertices):
        """
        Generate a mask for selecting the polygon region
        """
        mask = np.zeros_like(img)
        if len(img.shape) > 2:
            channel_count = img.shape[2]
            mask_color = (255,) * channel_count
        else:
            mask_color = 255
        # fill the pixels inside the polygon  
        cv2.fillPoly(mask, vertices, mask_color)
        return mask
    
    masked_image = cv2.bitwise_and(img, _generate_mask(img, vertices))
    return masked_image

def smooth(img, kernel_size=5):
    """
    Aplly a Gaussian blur to the image for smoothing
    """
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)